In [1]:
import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

from surprise import SVD

from collections import defaultdict

from surprise.model_selection import GridSearchCV

from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import SVD, SVDpp, NMF
from surprise import SlopeOne, CoClustering
from surprise import BaselineOnly

import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, dash_table

from PIL import Image
import requests
from io import BytesIO

import dash_bootstrap_components as dbc

import joblib

C:\Users\peter\Anaconda3\envs\geopandas_testenv\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#get game info
folder = r"C:\Users\peter\Desktop\tdi_capstone"
game_info_df = pd.read_csv(folder + "\\bgg_game_info.csv", index_col = 0)

In [3]:
#get ratings and comments
folder = r"C:\Users\peter\Desktop\tdi_capstone\bgg_api_data"

#read in master game csv
ratings_df = pd.read_csv(folder + "\\bgg_all_games.csv", index_col = 0)
ratings_df = ratings_df.reset_index(drop=True)

C:\Users\peter\Anaconda3\envs\geopandas_testenv\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
game_info_df.head()

,bgg_id,name,bgg_type,thumbnail,image,description,year_published,players_min,players_max,playingtime,...,boardgameartist,boardgamecategory,boardgamedesigner,boardgamefamily,boardgamemechanic,boardgamepublisher,boardgameimplementation,boardgameexpansion,boardgamecompilation,boardgameintegration
0,1,Die Macher,boardgame,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,Die Macher is a game about seven sequential po...,1986,3,5,240,...,"['Marcus Gschwendtner', 'Harald Lieske']","['Economic', 'Negotiation', 'Political']",['Karl-Heinz Schmiel'],"['Country: Germany', 'Political: Elections', '...","['Alliances', 'Area Majority / Influence', 'Au...","['Hans im Glück', 'Moskito Spiele', 'Ediciones...",NaN,NaN,NaN,NaN
1,2,Dragonmaster,boardgame,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,Dragonmaster is a trick-taking card game based...,1981,3,4,30,...,['Bob Pepper'],"['Card Game', 'Fantasy']","['G. W. ""Jerry"" D\'Arcey']","['Components: Gems/Crystals', 'Creatures: Drag...",['Trick-taking'],"['E.S. Lowe', 'Milton Bradley']","['Indulgence', ""Coup d'État""]",NaN,NaN,NaN
2,3,Samurai,boardgame,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,Samurai is set in medieval Japan. Players comp...,1998,2,4,60,...,['Franz Vohwinkel'],"['Abstract Strategy', 'Medieval']",['Reiner Knizia'],['Components: Map (Continental / National scal...,"['Area Majority / Influence', 'Hand Management...","['Fantasy Flight Games', 'Hans im Glück', '999...",['Samurai: The Card Game'],NaN,NaN,NaN
3,4,Tal der Könige,boardgame,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,When you see the triangular box and the luxuri...,1992,2,4,60,...,['Thomas di Paolo'],['Ancient'],['Christian Beierer'],"['Containers: Triangular Boxes', 'Country: Egy...","['Action Points', 'Area Majority / Influence',...",['KOSMOS'],NaN,NaN,NaN,NaN
4,5,Acquire,boardgame,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,"In Acquire, each player strategically invests ...",1964,2,6,90,...,"['Scott Okumura', 'Peter Whitley']","['Economic', 'Territory Building']",['Sid Sackson'],['Series: 3M Bookshelf Series'],"['Hand Management', 'Investment', 'Market', 'S...","['3M', 'The Avalon Hill Game Co', 'Avalon Hill...",NaN,['1963 World Map Conversion Kit for the Game o...,NaN,NaN


In [5]:
game_info_df['complexity_average'].max()

4.7378

In [6]:
ratings_df = ratings_df.merge(game_info_df[['bgg_id', 'name']], left_on='Bggid', right_on='bgg_id')

In [7]:
ratings_df.head()

,Username,Rating,Comment,Bggid,bgg_id,name
0,jackcres,10.0,NaN,1,1,Die Macher
1,FuManchu,10.0,Play this game. Play this game a lot.,1,1,Die Macher
2,HankM,10.0,hard to find enough players,1,1,Die Macher
3,jonnypolite,10.0,NaN,1,1,Die Macher
4,pwotruba,10.0,the older Moskito version is for 4 players onl...,1,1,Die Macher


In [8]:
ratings_df['Username'].nunique()

231893

In [9]:
ratings_df['name'].nunique()

856

In [10]:
ratings_df.shape

(2128849, 6)

In [11]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings_df[['Username', 'name', 'Rating']], reader)

In [ ]:
#We can now use this dataset as we please, e.g. calling cross_validate
#cross_validate(NormalPredictor(), data, cv=3)

In [12]:
#normal_cv
#normal_cv = cross_validate(NormalPredictor(), data, cv=3)
#normal_cv

In [15]:
baseline_cv = cross_validate(BaselineOnly(), data, cv=3)
baseline_cv

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.33483736, 1.33358861, 1.33455199]),
 'test_mae': array([0.99683461, 0.99596838, 0.99699407]),
 'fit_time': (10.011044025421143, 11.09975004196167, 11.340474843978882),
 'test_time': (9.283721923828125, 8.951181650161743, 9.844559669494629)}

In [ ]:
#k-NN Based Algorithms
#knnbasic_cv = cross_validate(KNNBasic(), data, cv=3)

In [ ]:
#knnmeans_cv = cross_validate(KNNWithMeans(), data, cv=3)

In [ ]:
#knnz_cv = cross_validate(KNNWithZScore(), data, cv=3)

In [16]:
#Matrix Factorization Based Algorithms
svd_cv = cross_validate(SVD(), data, cv=3)
svd_cv

{'test_rmse': array([1.31127976, 1.30904983, 1.30836479]),
 'test_mae': array([0.96520353, 0.96277543, 0.96151732]),
 'fit_time': (110.78350067138672, 113.63709616661072, 118.19681739807129),
 'test_time': (11.547534704208374, 10.838301420211792, 9.812180757522583)}

In [ ]:
#Other Collaborative Filtering Algorithms
#slope_cv = cross_validate(SlopeOne(), data, cv=5, n_jobs=5, verbose=False)
#slope_cv

In [12]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [19]:

param_grid =   {'reg_all': [0.002, 0.02, 0.08]}           #{'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=2)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.333510502071252
{'reg_all': 0.02}


In [13]:
# First train an SVD algorithm on the movielens dataset.
data = data
trainset = data.build_full_trainset()
#algo =   SVD()   #NormalPredictor()
#algo.fit(trainset)

#SVD
#NormalPredictor
#BaselineOnly

In [14]:
algo =   SVD()   #NormalPredictor()
algo.fit(trainset)


In [15]:
import joblib

In [ ]:
filename = 'recommender_model.sav'


In [ ]:
joblib.dump(algo, filename)

In [ ]:
loaded_model = joblib.load(filename)
#result = loaded_model.score(X_test, Y_test)

In [ ]:
algo = loaded_model

In [16]:
algo

In [17]:
user_df = pd.DataFrame(ratings_df['Username'].unique())
user_df = user_df.rename(columns={0: "user"})

In [18]:
fillValue =  trainset.global_mean #None

In [19]:
features_of_interest = ['name','bgg_id' ,'players_min', 'players_max', 'playingtime','rating_average' ,'complexity_average','boardgamecategory','boardgamemechanic', 'thumbnail']

In [20]:
features_df = game_info_df[features_of_interest]

In [21]:
def get_top_x_recs(username,x_recs, players_min, players_max, time_min, time_max, complexity_min, complexity_max):

    target_username = username
    
    targetUser = trainset.to_inner_uid(target_username) #inner_id of the target user

    anti_testset_user = []

    user_item_ratings = trainset.ur[targetUser]
    user_items = [item for (item,_) in (user_item_ratings)]
    ratings = trainset.all_ratings()

    for iid in trainset.all_items():
        if(iid not in user_items):
            anti_testset_user.append((trainset.to_raw_uid(targetUser),trainset.to_raw_iid(iid),fillValue))

    predictions = algo.test(anti_testset_user)

    pred = pd.DataFrame(predictions)

    pred.sort_values(by=['est'],inplace=True,ascending = False)

    pred = pred.rename(columns={"uid": "user", "iid": "name", "r_ui": "dataset_average", "est": "predicted_rating"})

    pred = pred.drop(columns=["details"])

    pred = pred.merge(features_df, left_on = "name", right_on = "name")
    
    pred = pred[pred['players_max'] <= players_max]
    
    pred = pred[pred['players_max'] >= players_min]
    
    
    pred = pred[pred['playingtime'] >= time_min]
    
    pred = pred[pred['playingtime'] <= time_max]
    
    
    
    pred = pred[pred['complexity_average'] >= complexity_min]
    
    pred = pred[pred['complexity_average'] <= complexity_max]
    
    pred = pred.round({'predicted_rating': 1, 'rating_average' : 1})
    
    
    top_x = pred[:x_recs]

    return top_x

In [22]:
def get_image(dff, x_iloc):
    image_url = dff['thumbnail'].iloc[x_iloc]
    
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    return img

In [23]:
def range_slider_transform(value):
    if value == 0:
        return 0
    if value < 5:
        return 2 ** (value -1) * 30
    if value ==5:
        return 99999

In [24]:
def make_link(dff, loc):
    
    web_address = 'https://boardgamegeek.com/boardgame/'
    
    game_name = dff['name'].iloc[loc]
    if len(game_name) > 20:
        
        game_name = game_name[:17] + '...'
    game_web =  web_address + str(dff['bgg_id'].iloc[loc])
    
    return  html.A(game_name, href= game_web)

In [25]:



#app = Dash(__name__)
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])
#server = app.server




controls = dbc.Card([
    
    html.H4("First Pick Game: A Board Game Recommender System"),
    
    dbc.Card([
    
    dbc.Label(["User for game recommendations"]),
    
    
    html.Div([
        dcc.Dropdown(user_df.user[:500],user_df.user[0], id='pandas-dropdown-2'),
   #     html.Div(id='pandas-output-container-2')
    ]),
    ],body= True, outline= True),

    dbc.Card([
    
    dbc.Label(["Number of players in game"]),
    
    dcc.RangeSlider(min=1, max=10, step=1, value=[1, 10], id='my-range-slider'),
    ],body= True, outline= True),    
    
    
    dbc.Card([
    
    dbc.Label(["Average game time"]),
    
    dcc.RangeSlider(min=0, max=5, 
                    
                    marks={
        0: {'label': '0 min'},
        1: {'label': '30 min'},
        2: {'label': '1hr'},
        3: {'label': '2hr'},
        4: {'label': '4hr'},
        5: {'label': 'Unlimted'}
    },
                    
                    step=1, value=[0, 5], id='my-range-slider2'),
    ],body= True, outline= True), 
        
    dbc.Card([
    dbc.Label(["Game complexity"]),
    
    dcc.RangeSlider(min=0, max=5, step=1, value=[0, 5], id='my-range-slider3'),
    ],body= True, outline= True), 
    
        
    ], body= True, outline= True, style={'backgroundColor': 'white'})





pictures =   dbc.Card([
    
        dbc.Label(["Top Five Recommended Games"]),
        dbc.Row([ 
            
        dbc.Col([
            dbc.Label(id = "Game1"  ),
            dbc.Card([   (html.Img(id = "image1", width="162", height="162")) ])], width=1.5),
            
        dbc.Col([
            dbc.Label(id = "Game2"  ),
            dbc.Card([   (html.Img(id = "image2", width="162", height="162")) ])], width=1.5),
            
        dbc.Col([
            dbc.Label(id = "Game3"  ),
            dbc.Card([   (html.Img(id = "image3", width="162", height="162")) ])], width=1.5),
            
        dbc.Col([
            dbc.Label(id = "Game4"  ),
            dbc.Card([   (html.Img(id = "image4", width="162", height="162")) ])], width=1.5),
            
        dbc.Col([
            dbc.Label(id = "Game5"  ),
            dbc.Card([   (html.Img(id = "image5", width="162", height="162")) ])], width=1.5),
            
                ])
        ], body= True, outline= False, style={'backgroundColor': 'white'})


table = dbc.Card([

dcc.Graph(id='indicator-graphic2'),
    

    ],body= True)


app.layout = dbc.Card(
    [
        
        
     #   dbc.Row(
     #       dbc.Col(navbar),
      #          no_gutters=True),
        
        
       # dbc.Row(
       #     [
       #         dbc.Col(html.H1("First pick game"), width=4),
                #dbc.Col(pictures, width = 8),
       #     ]),
        
       # html.Hr(),
        
        dbc.Row(
            [
                dbc.Col(controls, width= 3),
                
                dbc.Col([
                   
                 dbc.Col(   dbc.Row(pictures), lg={"size": 12, "offset": 2} ),
                    
                    dbc.Label(["Game Info"]),
                    
                    dbc.Row(table),], width = 8)
                         
            ],
            
        ),
        
        
      
        #dcc.Graph(id='indicator-graphic'),
        
    ]
    
)
        

 
 

@app.callback(
 #   Output('pandas-output-container-2', 'children'),
    Output('image1', 'src'),
    Output('image2', 'src'),
    Output('image3', 'src'),
    Output('image4', 'src'),
    Output('image5', 'src'),
    Output('Game1', 'children'),
    Output('Game2', 'children'),
    Output('Game3', 'children'),
    Output('Game4', 'children'),
    Output('Game5', 'children'),
    #Output('indicator-graphic', 'figure'),
    Output('indicator-graphic2', 'figure'),
    Input('pandas-dropdown-2', 'value'),
    Input('my-range-slider', 'value'),
    Input('my-range-slider2', 'value'),
    Input('my-range-slider3', 'value'))

    
    

def update_graph(value, range_values, range_values2, range_values3):
    
    #convert the time slider 
    range_values2 = [range_slider_transform(v) for v in range_values2]
    
   
    #get data frame based off slider values
    dff = get_top_x_recs(value, 5, range_values[0], range_values[1], range_values2[0],
                         range_values2[1], range_values3[0], range_values3[1])
    
    #get df for app table, by filtering on the columns that are displayed
    df_table = dff[['name', 'predicted_rating', 'rating_average', 'boardgamecategory', 'boardgamemechanic',
                   'players_min', 'players_max', 'playingtime', 'complexity_average' ]]
    
    table_list = df_table.values.tolist()
    table_list.insert(0,df_table.columns.tolist())
    
    table = go.Figure(data=[go.Table(
    #header=dict(values=list(df_table.columns),
    #            fill_color='white',
    #            align='left'),
    cells=dict(values=  table_list,       #df_table, #cells=dict(values=df_table.transpose().values.tolist(), # cells=dict(values=df_table.values.tolist(),
               fill_color='white',
               align='left'))
    ])
    
    table.update_layout(height=440, margin={"r":0,"t":0,"l":0,"b":0}) 
    
   
    #get image thumbnails
    img1 = get_image(dff, 0)
    img2 = get_image(dff, 1)
    img3 = get_image(dff, 2)
    img4 = get_image(dff, 3)
    img5 = get_image(dff, 4)
    
    
    
    
    #get hyperlinks for games to board game geek
    game1 = make_link(dff,0)
    game2 = make_link(dff,1)
    game3 = make_link(dff,2)
    game4 = make_link(dff,3)
    game5 = make_link(dff,4)
    
    return  img1, img2, img3, img4, img5, game1, game2, game3, game4, game5, table


    


if __name__ == '__main__':
    app.run_server(debug=True)
    #app.run_server(mode='inline')


Dash app running on http://127.0.0.1:8050/
